In [ ]:
# shape of x is number of samples x time steps(lookback) x number of features (type of data)
# shape of y is number of expected outcome x number of features (type of data)

In [1]:
# import libraries
import numpy as np

def create_sequence(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), :]
        X.append(a)
        Y.append(dataset[i + look_back, :])
    return np.array(X), np.array(Y)

In [15]:
# import libraries
import numpy as np

def create_pred_sequence(dataset, look_back=1):
    X = []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), :]
        X.append(a)
    return np.array(X)

In [2]:
# import libraries
import numpy as np
import pandas as pd
# select the data
def select_data(data, start=2016, end=2024, columns=None):
    # convert the data to a pandas dataframe
    data['Date'] = pd.to_datetime(data['Date'])
    # create a mask for the data
    if columns is None:
        mask = (data['Date'].dt.year >= start) & (data['Date'].dt.year<= end)
    else:
        mask = (data['Date'].dt.year >= start) & (data['Date'].dt.year<= end) & (data.columns.isin(columns))
    # select the data
    data = data.loc[mask]
    # drop the date column
    data = data.drop(columns='Date')
    return data

In [3]:
# import libraries
from sklearn.preprocessing import MinMaxScaler

def create_scaler(dataset):
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(dataset)
    return scaler, dataset 

In [4]:
# import libraries
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

def create_model(dataset, look_back=1, epochs=100, batch_size=1, verbose=0):
    X, Y = create_sequence(dataset, look_back)
    X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))
    Y = np.reshape(Y, (Y.shape[0], Y.shape[1]))
    
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
    #model.add(Dropout(0.2))
    model.add(LSTM(units=50))
    model.add(Dense(X.shape[2]))  # Assuming single feature for simplicity
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X, Y, epochs=epochs, batch_size=batch_size, verbose=verbose)
    
    return model

In [18]:
# import libraries
import numpy as np
def create_input_data(dataset, look_back=1):
    dataset = np.array(dataset)
    dataset = dataset[-look_back:]
    seq = create_pred_sequence(dataset, look_back)
    print(seq.shape)
    np.reshape(seq, (1, seq[0].shape[0], seq[0].shape[1]))
    return seq


In [6]:
# import libraries
def predict(model, dataset, look_back=1):
    # take last look_back days from dataset
    dataset = dataset[-look_back:]
    # create input datas
    X = create_input_data(dataset, look_back)
    # make prediction
    prediction = model.predict(X)
    return prediction

In [7]:
# import libraries
import numpy as np
def predict_next_days(model, dataset, look_back=1, days=1):
    predictions = []
    for i in range(days):
        prediction = predict(model, dataset, look_back)
        predictions.append(prediction)
        dataset = np.append(dataset, prediction)
    return predictions

In [8]:

# declare the path of the data
path_to_data = r"data\cleansing\filled\mean_filled.csv"

# load the data
# import pandas as pd
data = pd.read_csv(path_to_data)

# select the desired data
req_data = select_data(data=data)

In [9]:
# scale the data
scaler, scaled_data = create_scaler(req_data) # scaled_data is a numpy array

In [10]:
# split the data into training and testing
train_size = int(len(scaled_data) * 0.80)
test_size = len(scaled_data) - train_size
train, test = scaled_data[0:train_size], scaled_data[train_size:len(scaled_data)]

In [11]:
print(train.shape)

(2536, 22)


In [12]:
x_train, y_train = create_sequence(train)
# print shape of x_train and y_train
print(x_train.shape)
print(y_train.shape)
print(x_train)

(2535, 1, 22)
(2535, 22)
[[[0.062744   0.08675535 0.         ... 0.1292732  0.08383234 0.1611815 ]]

 [[0.06553263 0.13880856 0.02547575 ... 0.12640046 0.08662675 0.17992353]]

 [[0.06553263 0.13880856 0.02547575 ... 0.12640046 0.08662675 0.20241397]]

 ...

 [[0.65923034 0.62984384 0.68201351 ... 0.72191899 0.16670659 0.06747132]]

 [[0.65923034 0.64256796 0.68201351 ... 0.71186441 0.16518962 0.06664668]]

 [[0.64500837 0.6437247  0.67771639 ... 0.7075553  0.16071856 0.06934553]]]


In [13]:
# create the model
model = create_model(train, look_back=1, epochs=100, batch_size=1, verbose=0)

(2535, 1, 22)
(2535, 22)


c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [19]:
# make prediction
predictions = predict_next_days(model, test, look_back=1, days=1)

(0,)


IndexError: index 0 is out of bounds for axis 0 with size 0